### LangChain Examples Notebook: Agents


IMPORTS

In [1]:
import torch
from ais_utils.Model_from_LC_Ollama import get_chatLLM, get_chatOpenAI, get_LLM

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
llm = get_chatOpenAI() # get_chatLLM()

In [3]:
#%pip install -qU duckduckgo-search langchain-community

In [4]:
# custom search tool
from langchain_community.tools import DuckDuckGoSearchResults

search_tool = DuckDuckGoSearchResults(backend='news')

# testing tool
search_tool.invoke("What is breaking news headlines for today")

'[snippet: Azuradee France, 33, was sentenced to 35 to 60 years in prison during an emotional and tense court hearing on Monday., title: Detroit mother gets 35+ years in prison for death of 3-year-old son found in freezer, link: https://www.msn.com/en-us/news/crime/detroit-mother-gets-35-years-in-prison-for-death-of-3-year-old-son-found-in-freezer/ar-BB1qQzER, date: 2024-07-29T23:22:30+00:00, source: USA Today on MSN.com], [snippet: Not a justice on the Supreme Court of the United States. No one."Start the day smarter. Get all the news you need in your inbox each morning.             Of note: Biden did not call for expanding the Supreme Court − a proposal that is popular among progressives as a way of counteracting the current conservative supermajority., title: \'No one is above the law\': Biden calls for sweeping Supreme Court changes, link: https://www.msn.com/en-us/news/politics/no-one-is-above-the-law-biden-calls-for-sweeping-supreme-court-changes/ar-BB1qOrJl, date: 2024-07-29T22:

In [5]:
from langchain.tools import BaseTool

# Custom Calculator Tool
class Calculator(BaseTool):
    name = "calculator"
    description = "Useful for when you need to perform mathematical calculations."

    def _run(self, query: str) -> str:
        return str(eval(query))

calculator_tool = Calculator()

#testing tool
calculation_result = calculator_tool.run("2 + 2 * 3")
print(calculation_result)

8


In [6]:
from langchain.agents import Tool

# Define tools for the agents
tools = [
    Tool(
        name="Search",
        func=search_tool.run,
        description="useful for when you need to answer questions about current events or the current state of the world",
    ),
    Tool(
        name="Calculator", 
        func=calculator_tool.run,
        description="useful for when you need to perform mathematical calculations",
    )
]

tool_names =  [tool.name for tool in tools]

In [7]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [8]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...
Sure, here's the solution:

3^5 * (12 + 3)^2 = 243 * 15 = 360

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': "Sure, here's the solution:\n\n3^5 * (12 + 3)^2 = 243 * 15 = 360"}

In [10]:
agent_executor.invoke(
    {
        "input": "Was Eines Stein born earlear than Winston Churchill?, or was it named after the German city of Wesel?"
    }
)



> Entering new AgentExecutor chain...
The context does not provide any information about the birthdate or origin of Eines Stein, so I cannot answer this question from the provided context.

> Finished chain.


{'input': 'Was Eines Stein born earlear than Winston Churchill?, or was it named after the German city of Wesel?',
 'output': 'The context does not provide any information about the birthdate or origin of Eines Stein, so I cannot answer this question from the provided context.'}